In [1]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
import pickle
import pandas as pd

In [2]:
battery_names_file = open("battery_names_file.pkl", "rb")
bat_dict = pickle.load(battery_names_file)

num_cycles = np.loadtxt('num_cycles.csv', dtype=int)

test_ind = np.loadtxt('test_ind.csv', dtype=int, delimiter=',')
train_ind = np.loadtxt('train_ind.csv', dtype=int, delimiter=',')
secondary_test_ind = np.loadtxt('secondary_test_ind.csv', dtype=int, delimiter=',')

In [115]:
# dict_c = {}
# keys = range(124)
# for i in keys:
#     dict_c[i] = num_cycles[i]

# for key, value in dict_c.items():
#     print(key, ' : \n', value)

In [ ]:
F_all_batt_all_c = []

for batt in range(0, len(bat_dict.keys())):
    
    #if batt ==63:
    #    continue     
        
    features_data = pickle.load(open(r'./F_tot_batt{}.pkl'.format(batt),'rb'))
    
    cycle_num = len(features_data)    
    features_size = len(features_data[0][0])
    count = 0
    F_1c = np.zeros((1, features_size))
    F_allc = np.zeros((1, features_size))   #fictious value
    Q_allc = []
    
    #if batt == 42 or batt ==63 or batt ==65:

    for c in range(0, cycle_num):
    #for c in range(0, 200):
        
        if len(features_data[c][0][:, 1])<68:
            F_1c[0, :24] = features_data[c][0][:24, 1]
            F_1c[0, -24:] = features_data[c][0][-24:, 1]
            F_1c[0, 24:-24] = np.linspace(features_data[c][0][23, 1], features_data[c][0][-24, 1], 20) 
            count+=1
        else:
            F_1c = features_data[c][0][:, 1].reshape(1, -1)

        Q_1c = features_data[c][1]
        t_max_1c = features_data[c][2]
        
        F_allc = np.r_[F_allc, F_1c]
        Q_allc.append(Q_1c)

        #F is a list 
        #column 0 contains cycle# arrays of 68 features (call F_tot[0][cycle])
        #2nd column contains Q F_tot[1][cycle]
    
    F_tot = [F_allc[1:, :], Q_allc]       #remove first row of F_all_c that was fictious
    F_all_batt_all_c.append(F_tot)
    print('batt #', batt)
    print('# exceptions:', count)  # count cases in which #Features<68
    
with open("F_all_batt_all_c", "wb") as fp:   #Pickling
    pickle.dump(F_all_batt_all_c, fp)

In [4]:
## remove outliers from F_all_batt_all_c 
## the criteria adopted is take windows of 10 points and use hampel filtering

In [ ]:
with open("F_all_batt_all_c", "rb") as fp:   # Unpickling
    F_all_batt_all_c = pickle.load(fp)
    
F_all_batt_clean = F_all_batt_all_c.copy()
for idx in range(0, len(F_all_batt_clean)):
#for idx in range(8, 9):
    V_feature_tot = F_all_batt_all_c[idx][0].copy()
    Q_feature_tot = np.array(F_all_batt_all_c[idx][1]).reshape(-1, 1).copy()
    batch_i_tot = np.c_[Q_feature_tot, V_feature_tot]   # copy capacities in position 1 and voltage features in position 2

    F_old = batch_i_tot.copy()    
    
    window_size= 10   
    ## this modify the tail in features where there is a spike at the end of curve
    
    for kk in range(0, batch_i_tot.shape[1]):
        df = pd.Series(batch_i_tot[:, kk])
        pos = hampel(df, window_size, n=1) 
        windows = df.rolling(window_size, min_periods=1)
        moving_averages = windows.median().tolist()
        for ii in range(0, len(pos)):
            batch_i_tot[pos[ii], kk] = np.array(moving_averages[pos[ii]])
    
    for kk in range(0, batch_i_tot.shape[1]):
        plt.figure()
        plt.plot(np.linspace(0, 1, len(F_old)), F_old[:, kk], 'k.')
        plt.plot(np.linspace(0, 1, len(batch_i_tot)), batch_i_tot[:, kk], 'r.')
        #plt.ylim(batch_i_tot[:, kk].min(), batch_i_tot[:, kk].max())
        plt.title('batt{}_F{}'.format(idx, kk))

    F_all_batt_clean[idx][0] = batch_i_tot[:, 1:-1]
    F_all_batt_clean[idx][1] = batch_i_tot[:, 0]
    
    for kk in range(0, batch_i_tot.shape[1]):
        plt.figure()
        plt.plot(np.linspace(0, 1, len(F_old)), F_old[:, kk], 'b.')
        plt.plot(np.linspace(0, 1, len(batch_i_tot)), batch_i_tot[:, kk], 'g.')
        #plt.ylim(batch_i_tot[:, kk].min(), batch_i_tot[:, kk].max())
        plt.title('batt{}_F{}'.format(idx, kk))
        
with open("F_all_batt_clean", "wb") as fp:   #Pickling
    pickle.dump(F_all_batt_clean, fp)

In [ ]:
### see how the features are with different batteries in train_dataset

In [30]:
import pickle
import numpy as np
import pandas as pd
from numpy import genfromtxt

import matplotlib.pyplot as plt

import sys
import os
from pathlib import Path
import numpy as np
import math

from scipy.integrate import odeint

import torch
import torch.nn as nn

In [ ]:
# discard batteries 41, 58, 59
# open num_cycleswIv4

print(train_ind, 'train_ind')
print(test_ind, 'test_ind')
print(secondary_test_ind, 'secondary_test_ind')
#train_ind = np.delete(train_ind, np.where(train_ind==41)[0])
#train_ind = np.delete(train_ind, np.where(train_ind==57)[0])    ## this is wrong.. batt 59 correspond to new idx 57 in F_all_batt_clean_wI4
#test_ind = np.delete(test_ind, np.where(test_ind==58)[0])  # batt 58 correspond to idx 57 in F_all_batt_clean_wI4
print(train_ind, 'train_ind')
print(test_ind, 'test_ind')
secondary_test_ind = np.delete(secondary_test_ind, -1)
secondary_test_ind = np.delete(secondary_test_ind, -1)
secondary_test_ind = np.delete(secondary_test_ind, -1)
secondary_test_ind = np.delete(secondary_test_ind, -1)
print(secondary_test_ind, 'secondary_test_ind')


num_cycles = np.delete(num_cycles, 41)
num_cycles = np.delete(num_cycles, 57)
num_cycles = np.delete(num_cycles, 57)


## collect C rates
dt = np.dtype(object)
charge_policy_list = np.loadtxt('./charge_policy_dict.csv', dtype=dt, delimiter=',', skiprows=1)
charge_policy_list = np.delete(charge_policy_list, 41)   
charge_policy_list = np.delete(charge_policy_list, 57) 
charge_policy_list = np.delete(charge_policy_list, 57)

C1_list = []
C2_list = []
P1_list = []

for idx in range(0, len(charge_policy_list)):
  #C_rate = bat_dict[idx]['charge_policy']
    C_rate_i = charge_policy_list[idx]
    print(C_rate_i)
    C_rate_list = re.findall(r'-?\d+\.?\d*', C_rate_i)
        
    C1 = np.fromstring(C_rate_list[0], dtype='float64', sep=" ")
    P1 = np.fromstring(C_rate_list[1], dtype='float64', sep=" ")
    C2 = -np.fromstring(C_rate_list[2], dtype='float64', sep=" ")

    C1_list.append(C1)
    C2_list.append(C2)
    P1_list.append(P1)

print(' C1 max is:', np.array(C1_list).max())
print(' C1 min is:', np.array(C1_list).min())
print(' C2 max is:', np.array(C2_list).max())
print(' C2 min is:', np.array(C2_list).min())
print(' P1 max is:', np.array(P1_list).max())
print(' P1 min is:', np.array(P1_list).min())

In [ ]:
with open("./../../data/F_all_batt_clean", "rb") as fp:   # Unpickling
    F_all_batt_all_c = pickle.load(fp)

list_element_key = lambda x: list(bat_dict)[x]
train_key_list = [list_element_key(x) for x in(train_ind)]
valid_key_list = [list_element_key(x) for x in(test_ind)]
train2_key_list = [list_element_key(x) for x in(secondary_test_ind)]

N_train = len(train_key_list)
print(N_train, 'N_train')
N_test = len(valid_key_list)
print(N_test, 'N_test')
N_train_2 = len(train2_key_list)
print(N_train_2, 'N_train_2')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
def make_name(dict):
    opsep = '_'
    name = 'LSTM20p2'
    for key, value in dict.items():
        name += '{}'.format(key)+opsep+'{}'.format(value)+opsep
    return name


class LogDataset_():
    def __init__(self, idx_list, F_all_batt, sampling_freq, Crate1, Crate2, Prate, load_mode): 
        self.idx_list = idx_list
        self.load_mode = load_mode
        self.F_all_batt = F_all_batt
        self.Crate1 = Crate1
        self.Crate2 = Crate2
        self.Prate = Prate
               
    def __len__(self):
        return len(self.idx_list)
          
    def __getitem__(self, idx):
        
        V_feature_tot = np.delete(self.F_all_batt[self.idx_list[idx]][0], [0, 25, 37, 44, 45, -1], 1)
        Q_feature_tot = np.array(self.F_all_batt[self.idx_list[idx]][1]).reshape(-1, 1)
        lendata = tmax_feature_tot.shape[0]

        batch_i_tot = np.c_[Q_feature_tot, V_feature_tot]
        
        #here we normalize the features
        batch_i_min = batch_i_tot.min(axis = 0).reshape(1, -1)
        batch_i_max = batch_i_tot.max(axis = 0).reshape(1, -1)
        batch_i_norm = (batch_i_tot-batch_i_min)/(batch_i_max-batch_i_min)
        C1_norm = (self.Crate1[self.idx_list[idx]]-1)/7
        C2_norm = (self.Crate2[self.idx_list[idx]]-3)/3
        P_norm = (self.Prate[self.idx_list[idx]]-2)/78

        #here we add the charge policy into the set of features
        batch_i_n = np.c_[batch_i_norm, C1_norm.repeat(lendata).reshape(-1, 1), C2_norm.repeat(lendata).reshape(-1, 1), P_norm.repeat(lendata).reshape(-1, 1)]
        #here we select only the features from feature sub-sampling
        select_features = [0, 1, 2, 3, 5, 6, 8, 10, 12, 14, 20, 22, 23, 24, 26, 28, 29, 30, 36, 37, 41, 64, 65, 66]      #CM Re Final
        select_features = [ 0,  4,  5,  9, 11, 12, 14, 21, 22, 25, 30, 33, 35, 36, 37, 39, 44, 48, 55, 56, 58, 59, 64, 66]   # Sal Re Final

        batch_i_ss = batch_i_n[::(sampling_freq//1), :]      #with sampling frequency !=1 we can select the frequency of data points along cycles 
        
        print(batch_i_ss.shape)
         
        if self.load_mode == 'all':
            batch_i_s = batch_i_ss[::1, :]
            true_y = torch.zeros(2973//sampling_freq, batch_i_s.shape[1], dtype=torch.float32)                               #e
            batch_i_s_torch = torch.from_numpy(batch_i_s).type(torch.float32)
            if len(batch_i_s_torch)==2973//sampling_freq+1:
                true_y[:len(batch_i_s_torch)-1, :] = batch_i_s_torch[:-1, :]
                batch_y = torch.unsqueeze(true_y, 0)    # (#batt, #steps, #features)
                pad = len(batch_i_s)-1
            else:
                true_y[:len(batch_i_s_torch), :] = batch_i_s_torch
                batch_y = torch.unsqueeze(true_y, 0)    # (#batt, #steps, #features)
                pad = len(batch_i_s)
                   
        return batch_y, pad

In [ ]:
# Set parameters

def makedirs(dirname):
    if not os.path.exists(dirname):
        os.makedirs(dirname)
                    
os.system('clear')

sampling_freq = 1
feature_size = 67 #67, #24, #25, #64

list_element_key = lambda x: list(bat_dict)[x]
train_key_list = [list_element_key(x) for x in(train_ind)]
valid_key_list = [list_element_key(x) for x in(test_ind)]
test_key_list = [list_element_key(x) for x in(secondary_test_ind)]

N_train = len(train_ind)+N_train_2

data_size = 2973//sampling_freq

## Set parameters
# Data params
batch_size = data_size
test_size = data_size

# Network params
input_size = feature_size
output_dim = feature_size                      
dtype = torch.float32

N_train = len(train_ind)+N_train_2
N_valid = int(0.1*len(test_ind))
N_test = len(test_ind)-N_valid

valid_idx, test_idx = random_split(test_ind, [N_valid, N_test], generator=torch.Generator().manual_seed(1))
a = DataLoader(valid_idx, len(valid_idx), shuffle=False)
b = DataLoader(test_idx, len(test_idx), shuffle=False)
for i, idx in enumerate(a):
    v_indx = idx.detach().numpy()
for i, idx in enumerate(b):
    t_indx = idx.detach().numpy()

all_ind = np.append(train_ind, secondary_test_ind)  
all_ind = np.append(all_ind, v_indx)
all_ind = np.append(all_ind, t_indx)

all_ind_rep = np.append(all_ind[:85].repeat(2), all_ind[85:])    #here we repeat the training batteries twice to train more data
train_all = LogDataset_(all_ind_rep, F_all_batt_all_c, sampling_freq, C1_list, C2_list, P1_list, load_mode ='all')
dataloader_all  = DataLoader(train_all, batch_size=len(train_all), shuffle=False)

# prepare the data
for i, n in enumerate(dataloader_all):
    train_all_input = n[0].squeeze(1)[:, :, :]
    train_all_target = n[0].squeeze(1)[:, 1:, :]
    L_all = n[1]+0.33*n[1]   #we add a plateau at the end of each dataset to allow longer predictions

L_all = torch.from_numpy(np.ceil(np.array(L_all))).type(torch.int32)

print(L_all, 'L_all')
print(len(L_all), 'len stacked data')


from scipy import interpolate

N_interp = 100 
N_b = len(L_all)

train_interp = train_all_input.clone() 
N_b, N_max_step, N_F = train_interp.shape 
train_interp_w_time = torch.empty((N_b, N_interp, N_F+1), dtype = torch.float32) 
target_interp_w_time = torch.empty((N_b, N_interp, N_F+1), dtype = torch.float32)

n_max = 0.3 # value to optimize 
n_pred = 4

random_portions1 = np.random.uniform(low=0.1, high=n_max, size=(85+85,)) # range of interpolation of training data 
random_portions2 = np.random.uniform(low=0.1, high=0.3, size=(12,)) 
random_portions3 = np.random.uniform(low=0.2, high=0.2, size=(24,))
random_portions = np.concatenate((random_portions1, random_portions2, random_portions3)) 
print(random_portions, 'random_portions')

train_interp_pred = torch.empty((N_b, int(np.floor(N_interp*n_pred)), N_F+1), dtype = torch.float32) 
target_interp_pred = torch.empty((N_b, int(np.floor(N_interp*n_pred)), N_F+1), dtype = torch.float32)

for ii in range(0, N_b):

    L_ii = int(L_all[ii])
    L_partial = np.floor(random_portions[ii]*L_ii/1.33) 
    L_pred = np.ceil(L_partial*n_pred)

    print(L_ii, 'L_ii')
    print(L_partial, 'L_partial')
    print(random_portions[ii], 'random_portions[ii]')
    
    max_L = 2234
    
    delta = int(L_partial)/N_interp
    t_N_tilde = torch.arange(0, int(L_partial), delta, dtype = torch.float32)/max_L
    t_tilde_RUL = torch.arange(0, L_pred, delta, dtype = torch.float32)/max_L

    train_interp_w_time[ii, :, 0] = t_N_tilde[:N_interp]
    target_interp_w_time[ii, :, 0] = train_interp_w_time[ii, :, 0]+1/max_L

    train_interp_pred[ii, :, 0] = t_tilde_RUL[:int(np.floor(N_interp*n_pred))].type(torch.float32)
    target_interp_pred[ii, :, 0] = train_interp_pred[ii, :int(np.floor(N_interp*n_pred)), 0]+1/2234

    t_N_tilde_target = target_interp_w_time[ii, :N_interp, 0].type(torch.float32).tolist()
    t_tilde_RUL_target = target_interp_pred[ii, :int(np.floor(N_interp*n_pred)), 0].type(torch.float32).tolist()

    for k in range(0, N_F):
        if L_ii<= 2972:
            y_3 = train_interp[ii, :L_ii+1, k].reshape(-1)   ## this is the vector containing all data
            x_3 = torch.arange(0, L_ii+1, 1)/max_L                  ## vector of cycles until EoL
        elif L_ii> 2972:
            y_3 = train_interp[ii, :L_ii, k].reshape(-1)   ## this is the vector containing all data
            x_3 = torch.arange(0, L_ii)/max_L         
        else:
            y_3 = train_interp[ii, :L_ii+1, k].reshape(-1)   ## this is the vector containing all data
            x_3 = torch.arange(0, L_ii+1, 1)/max_L                  ## vector of cycles until EoL
            
        #print(t_N_tilde_target, 't_N_tilde')
        #print(t_tilde_RUL_target, 't_tilde_RUL')

        interpolate_func = interpolate.interp1d(x_3, y_3, kind='cubic')

        ## derive the interpolated function (train, target) until random portion
        qinterp_train = (interpolate_func(t_N_tilde[:N_interp])).reshape(-1)
        train_interp_w_time[ii, :, k+1] = torch.from_numpy(qinterp_train.astype(np.float32))       

        qinterp_target = (interpolate_func(t_N_tilde_target)).reshape(-1)
        target_interp_w_time[ii, :, k+1] = torch.from_numpy(qinterp_target.astype(np.float32))   

        ## derive the interpolated function (train, target) for forecast, from random portion on

        qinterp_train_pred = (interpolate_func(t_tilde_RUL[:int(np.floor(N_interp*n_pred))])).reshape(-1)
        train_interp_pred[ii, :, k+1] = torch.from_numpy(qinterp_train_pred.astype(np.float32))          

        qinterp_target_pred = (interpolate_func(t_tilde_RUL_target[:int(np.floor(N_interp*n_pred))])).reshape(-1)
        target_interp_pred[ii, :, k+1] = torch.from_numpy(qinterp_target_pred.astype(np.float32))   